# Challenge Part 2: Customer Narrow their travel search based on Temperature and Precipitation


In [6]:
import pandas as pd
import requests
import gmaps

from config import g_key



In [7]:
city_data_df = pd.read_csv("data/WeatherPy_Database.csv")
city_data_df

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,korla,5,CN,clear sky,18,41.7597,86.1469,85.05,0.0,0,13.69
1,1,jamestown,75,US,broken clouds,67,42.0970,-79.2353,77.29,0.0,0,5.99
2,2,airai,52,TL,broken clouds,62,-8.9266,125.4092,61.07,0.0,0,3.69
3,3,fort nelson,43,CA,scattered clouds,43,58.8053,-122.7002,71.62,0.0,0,2.66
4,4,victoria,98,HK,overcast clouds,90,22.2855,114.1577,88.97,0.0,0,1.01
...,...,...,...,...,...,...,...,...,...,...,...,...
565,565,saint-pierre,75,RE,broken clouds,83,-21.3393,55.4781,67.68,0.0,0,16.11
566,566,bhainsdehi,100,IN,overcast clouds,89,21.6467,77.6325,70.29,0.0,0,8.28
567,567,valenca,28,BR,scattered clouds,93,-13.3703,-39.0731,69.26,0.0,0,1.50
568,568,sept-iles,62,CA,broken clouds,80,50.2001,-66.3821,59.72,0.0,0,11.10


In [8]:
city_data_df.dtypes

City_ID                  int64
City                    object
Cloudiness               int64
Country                 object
Current Description     object
Humidity                 int64
Lat                    float64
Lng                    float64
Max Temp               float64
Rain (inches)          float64
Snow (inches)            int64
Wind Speed             float64
dtype: object

In [9]:
#configure my gmaps

gmaps.configure(api_key=g_key)

#Enter max temperature Criteria

min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?60
What is the maximum temperature you would like for your vacation?80


In [10]:
#Ask the Customer if you want it to be raining?
rain_amount = input ("Do you want it to be raining? (yes/no)")

Do you want it to be raining? (yes/no)no


In [11]:
snow_amount = input ("Do you want it to be snowing? (yes/no)")

Do you want it to be snowing? (yes/no)no


In [17]:
if rain_amount == "no" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
                                          
                                           
elif rain_amount == "no" and snow_amount == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_amount == "yes" and snow_amount == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0) &
                                          (city_data_df["Snow (inches)"] == 0)]
else: 
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] > 0.0) &
                                          (city_data_df["Snow (inches)"] > 0)]                                           

In [18]:
filtered_cities_df.head()

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
1,1,jamestown,75,US,broken clouds,67,42.0970,-79.2353,77.29,0.0,0,5.99
2,2,airai,52,TL,broken clouds,62,-8.9266,125.4092,61.07,0.0,0,3.69
3,3,fort nelson,43,CA,scattered clouds,43,58.8053,-122.7002,71.62,0.0,0,2.66
5,5,fayaoue,0,NC,clear sky,60,-20.6500,166.5333,71.29,0.0,0,18.61
6,6,sechura,10,PE,clear sky,70,-5.5569,-80.8222,72.03,0.0,0,18.25


In [19]:
filtered_cities_df.count()

City_ID                239
City                   239
Cloudiness             239
Country                239
Current Description    239
Humidity               239
Lat                    239
Lng                    239
Max Temp               239
Rain (inches)          239
Snow (inches)          239
Wind Speed             239
dtype: int64

In [21]:
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Lat", "Lng",]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,jamestown,US,77.29,42.0970,-79.2353,
2,airai,TL,61.07,-8.9266,125.4092,
3,fort nelson,CA,71.62,58.8053,-122.7002,
5,fayaoue,NC,71.29,-20.6500,166.5333,
6,sechura,PE,72.03,-5.5569,-80.8222,


In [27]:
params = {
    "radius":5000,
    "type":"lodging",
    "key": g_key
    
}

#Iterate through
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url,params=params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not Found... Skipping...")
    
    

Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...
Hotel Not Found... Skipping...


In [37]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,jamestown,US,77.29,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,airai,TL,61.07,-8.9266,125.4092,Juvinal Martin
3,fort nelson,CA,71.62,58.8053,-122.7002,The Fort Nelson Hotel
5,fayaoue,NC,71.29,-20.6500,166.5333,Chez Dydyce Guesthouse
6,sechura,PE,72.03,-5.5569,-80.8222,Hospedaje Costa Bella


In [38]:
output_data_file = "./data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [39]:
vacation_df = pd.read_csv("./data/WeatherPy_Vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,1,jamestown,US,77.29,42.0970,-79.2353,Hampton Inn & Suites Jamestown
1,2,airai,TL,61.07,-8.9266,125.4092,Juvinal Martin
2,3,fort nelson,CA,71.62,58.8053,-122.7002,The Fort Nelson Hotel
3,5,fayaoue,NC,71.29,-20.6500,166.5333,Chez Dydyce Guesthouse
4,6,sechura,PE,72.03,-5.5569,-80.8222,Hospedaje Costa Bella


In [42]:
# Using the Template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt>
<dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp} </dd>
"""
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat","Lng"]]

In [44]:
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))